<a href="https://colab.research.google.com/github/Haley-1617/agents/blob/master/Conversational_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
%pip install -q langchain langchain_experimental openai python-dotenv langchain_openai langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.6/426.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.3/233.3 kB 16.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.45.0 which is incompatible.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [26]:
from langchain_openai import ChatOpenAI
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories.in_memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [27]:
from google.colab import userdata
import os

# Add openai api key to environment variables
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")

In [32]:
# Load environment variables and initialize the language model
# openai
# llm = ChatOpenAI(model="gpt-3.5-turbo", max_tokens=1000, temperature=0)
# google gemini
llm = GoogleGenerativeAI(model="gemini-2.5-flash-lite")

In [14]:
# Create simple in-memory store for chat histories
store = {}

def get_chat_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [15]:
# Create prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

In [33]:
# Combine the prompt and model into a runnable chain
chain = prompt | llm

In [34]:
# Wrap the chain with message history
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key="input",
    history_messages_key="history"
)

In [36]:
session_id = "user123"

response1 = chain_with_history.invoke(
    {"input": "Hello! How are you?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response1)

response2 = chain_with_history.invoke(
    {"input": "What was my previous message?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response2)

AI: System: I am doing well, thank you for asking! I am always ready to assist you with your needs. How can I help you today?
AI: System: Your previous message was "Hello! How are you?".


In [37]:
# Print conversation history
print("\nConversation History:")
for message in store[session_id].messages:
    print(f"{message.type}: {message.content}")


Conversation History:
human: Hello! How are you?
ai: System: I am doing well, thank you for asking! I am always ready to assist you with your needs. How can I help you today?
human: Hello! How are you?
ai: System: I am doing well, thank you for asking! I am always ready to assist you with your needs. How can I help you today?
human: What was my previous message?
ai: System: Your previous message was "Hello! How are you?".
